**Import Libraries**

In [2]:
import pandas as pd
from geopy.geocoders import Nominatim

import numpy as np
import math

**Read Dataset**

In [3]:
#innovations = pd.read_parquet("sample_for_geocoding.parquet")
innovations = pd.read_parquet("geocoding_all_0425.parquet")

**Preprocessing (minimise time used for geocoding)**

In [4]:
#drop all duplicates
geocoded_innovations = innovations.drop_duplicates(subset = "user_location", keep ="first")
#drop all rows with lat/lon
geocoded_innovations = geocoded_innovations[geocoded_innovations["longitude"].isna() & geocoded_innovations["latitude"].isna()]
#only "user_location" column
geocoded_innovations = geocoded_innovations[["user_location"]]
#add empty "geocoded" column
geocoded_innovations["geocoded"] = np.nan

In [5]:
geocoded_innovations

,user_location,geocoded
index,,
6490,Paraguay,NaN
37741,Perú ツ,NaN
24976,bogota,NaN
130875,Ibagué,NaN
205594,"Santo Domingo, Rep. Dom.",NaN
...,...,...
96750,Eldorado-Misiones,NaN
108980,ɴᴇᴏ ᴄɪᴛʏ🏙️,NaN
133246,Parque Patricios,NaN


In [13]:
innovations

,created_at,twitter_id,user_id,user_location,longitude,latitude,place_box_1_long,place_box_1_lat,place_box_2_long,place_box_2_lat,country,country_code,place_full_name,place_id,place_name,inno_nr
index,,,,,,,,,,,,,,,,
6490,Sun Jan 01 08:10:59 +0000 2012,153387731521773569,191216844,Paraguay,NaN,NaN,None,None,None,None,None,None,None,None,None,inno_02
37741,Sun Jan 01 17:17:44 +0000 2012,153525325689655297,30079977,Perú ツ,NaN,NaN,None,None,None,None,None,None,None,None,None,inno_02
24976,Tue Jan 03 14:15:36 +0000 2012,154204266041585664,298356610,bogota,NaN,NaN,None,None,None,None,None,None,None,None,None,inno_02
130875,Sun Jan 08 02:41:19 +0000 2012,155841483243663360,285206171,Ibagué,NaN,NaN,None,None,None,None,None,None,None,None,None,inno_02
205594,Thu Feb 02 06:23:34 +0000 2012,164957110965243904,141093464,"Santo Domingo, Rep. Dom.",NaN,NaN,None,None,None,None,None,None,None,None,None,inno_02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112773,Sun Dec 08 00:02:56 +0000 2019,1203464944223686656,1094717144800153601,España,NaN,NaN,None,None,None,None,None,None,None,None,None,inno_25
133246,Sun Dec 08 02:38:07 +0000 2019,1203503997421838338,1080159674,Parque Patricios,NaN,NaN,None,None,None,None,None,None,None,None,None,inno_23
135290,Sun Dec 08 02:55:07 +0000 2019,1203508275595153409,883475939929329664,Venezuela,NaN,NaN,None,None,None,None,None,None,None,None,None,inno_20


**Geocoding**

In [7]:
import geocoder

In [8]:
unique_user_locations = geocoded_innovations["user_location"].tolist()
dict = {}

for location in unique_user_locations:
    result = geocoder.osm(location).json
    dict[location] = result





Status code Unknown from https://nominatim.openstreetmap.org/search: ERROR - HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://nominatim.openstreetmap.org/search: ERROR - HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://nominatim.openstreetmap.org/search: ERROR - HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://nominatim.openstreetmap.org/search: ERROR - HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://nominatim.openstreetmap.org/search: ERROR - HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://nominatim.openstreetmap.org/search: ERROR - HTTPSConnectionPool(host='no

In [9]:
#dict to df
geocoded_df = pd.DataFrame(dict)
geocoded_df = geocoded_df.transpose()

In [10]:
#left join innovation df and geocoded df
inno_geocoded = pd.merge(innovations, geocoded_df, left_on = "user_location", right_index= True, how="left")

In [11]:
#format created_at column
inno_geocoded["created_at"] = pd.to_datetime(inno_geocoded.created_at)
inno_geocoded["time"] = inno_geocoded["created_at"].dt.strftime('%d/%m/%Y, %H:%M:%S')

In [12]:
inno_geocoded

,created_at,twitter_id,user_id,user_location,longitude,latitude,place_box_1_long,place_box_1_lat,place_box_2_long,place_box_2_lat,...,district,housenumber,quarter,suburb,allotments,farm,hamlet,isolated_dwelling,island,time
index,,,,,,,,,,,,,,,,,,,,,
6490,2012-01-01 08:10:59+00:00,153387731521773569,191216844,Paraguay,NaN,NaN,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"01/01/2012, 08:10:59"
37741,2012-01-01 17:17:44+00:00,153525325689655297,30079977,Perú ツ,NaN,NaN,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"01/01/2012, 17:17:44"
24976,2012-01-03 14:15:36+00:00,154204266041585664,298356610,bogota,NaN,NaN,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"03/01/2012, 14:15:36"
130875,2012-01-08 02:41:19+00:00,155841483243663360,285206171,Ibagué,NaN,NaN,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"08/01/2012, 02:41:19"
205594,2012-02-02 06:23:34+00:00,164957110965243904,141093464,"Santo Domingo, Rep. Dom.",NaN,NaN,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"02/02/2012, 06:23:34"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112773,2019-12-08 00:02:56+00:00,1203464944223686656,1094717144800153601,España,NaN,NaN,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"08/12/2019, 00:02:56"
133246,2019-12-08 02:38:07+00:00,1203503997421838338,1080159674,Parque Patricios,NaN,NaN,None,None,None,None,...,NaN,NaN,NaN,Parque Patricios,NaN,NaN,NaN,NaN,NaN,"08/12/2019, 02:38:07"
135290,2019-12-08 02:55:07+00:00,1203508275595153409,883475939929329664,Venezuela,NaN,NaN,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"08/12/2019, 02:55:07"


In [13]:
#Export df to csv
inno_geocoded.to_csv ("inno_geocoded.csv", index = False, header=True)

In [89]:
#Store lat and lon in separate columns 
#geocoded_innovations['Lat'] = geocoded_innovations["geocoded"].apply(lambda x: x.latitude if x else None)
#geocoded_innovations['Lon'] = geocoded_innovations["geocoded"].apply(lambda x: x.longitude if x else None)

**Afterprocessing**

In [28]:
#drop "user_location" column before joining two dataset (dropping column, which can be found in both datasets (apart from index))
#geocoded_innovations.drop("user_location", axis=1, inplace = True)

#joining two dataframes (left join full dataset and geocoded subset)
#esult = innovations.join(geocoded_innovations)

#Populate NaN values in columns "geocoded", "Lat" and "Lon" of same locations
#result["geocoded"] = (result["geocoded"].fillna(result.groupby("user_location")["geocoded"].transform("first")))
#result["Lat"] = (result["Lat"].fillna(result.groupby("user_location")["Lat"].transform("first")))
#result["Lon"] = (result["Lon"].fillna(result.groupby("user_location")["Lon"].transform("first")))

#result



**Interactive Visualisation**

In [29]:
#import folium
#from folium.plugins import HeatMapWithTime
#from numpy import nan
#import math

In [30]:
#Removing all row, where Lat column has NaN values
#nonan = result[result["Lat"].notna()]

In [31]:
#lat_lon_list = []
#for i in nonan["created_at"].unique():
    #temp=[]
    #for index, instance in nonan[nonan["created_at"] == i].iterrows():
        #temp.append([instance["Lat"],instance["Lon"]])
    #lat_lon_list.append(temp)


In [33]:
#converting it to datetime format
#nonan["created_at"]= pd.to_datetime(nonan["created_at"])#creating a time index
#time_index = []
#for i in nonan["created_at"].unique():
    #time_index.append(i)#formatting the index
#date_strings = [d.strftime('%d/%m/%Y, %H:%M:%S') for d in time_index]

In [32]:
#m = folium.Map([47.32, 8.50], zoom_start=5, tiles = "CartoDB dark_matter", control_scale = True)

#HeatMapWithTime(lat_lon_list,radius=5,auto_play=True,position='bottomright',name="cluster",index=date_strings,max_opacity=1).add_to(m)

#m

In [ ]:
# Todo
# Geocode all dataframe
# add innovation selector to folium
# find better visualisation in folium. not heatmap but point
# Move coordinates for origninally geocoded tweets from columns "Latitude" and "Longitude" to "Lat" and "Lon"